User interface
==========

<font color='red'>**WARNING**</font> : All the part of this notebook are dependent from 1. If you want to use each of the 4 parts of the notebook separately without ever running 1 please uncomment each rospy.init_node() instruction

Also as a design choice execution of part 4 of this notebook stops part 2 therefore if you want to look at the plot after 4 please run it again. 

In [1]:
import rospy
import actionlib
import assignment_2_2022.msg
from geometry_msgs.msg import PoseStamped
from assignment_2_2022.msg import PlanningActionResult
import ipywidgets as widgets


client = actionlib.SimpleActionClient('/reaching_goal',assignment_2_2022.msg.PlanningAction)

In [2]:
def goal_callback(data):
    global goal_set_
    if data.status.status==2 or data.status.status==3:
        goal_set_=False

In [3]:
#same function as in client.py
def target_client(a,b):
    client.wait_for_server()
    o=PoseStamped()
    o.header.seq=0
    o.header.stamp.secs=0
    o.header.stamp.nsecs=0
    o.header.frame_id=""
    o.pose.position.x=a
    o.pose.position.y=b
    o.pose.position.z=0
    o.pose.orientation.x=0
    o.pose.orientation.y=0
    o.pose.orientation.z=0
    o.pose.orientation.w=0
    goal = assignment_2_2022.msg.PlanningGoal(o)
    client.send_goal(goal)

In [4]:
#Define 2 buttons
a=widgets.Button(
    description='Set goal', 
    layout=widgets.Layout(width='30%', height='50px'),
    diabled=False, 
    button_style='', 
    icon='check'
)

b=widgets.Button(
    description='Cancel previous goal?', 
    layout=widgets.Layout(width='30%',height='50px'),
    diabled=False, 
    button_style='', 
    icon='trash'
)

a.style.button_color='green'
b.style.button_color='red'

output=widgets.Output()
output2=widgets.Output()
output3=widgets.Output()

p=widgets.FloatText(value='0.0',disabled=False)
q=widgets.FloatText(value='1.0',disabled=False)

print("Insert goal x-axis")
display(p)
print("Insert goal y-axis")
display(q)
display(widgets.HBox([a,b]),widgets.HBox([output,output2]))


#callback for the buttons
def button_clicked_setgoal(x):
    global goal_set_
    if goal_set_==False:
        target_client(p.value,q.value)
        goal_set_=True
    else:
        with output:
            print("Cannot set the goal")
        
def button_clicked_goalcancelled(y):
    global goal_set_
    if goal_set_==True:
        client.cancel_all_goals()
        goal_set_=False
        



rospy.init_node('target_client_py')
client.cancel_all_goals() #In this way if user runs again no problem occours
global goal_set_
goal_set_ = False
client = actionlib.SimpleActionClient('/reaching_goal',assignment_2_2022.msg.PlanningAction)
P=rospy.Subscriber('/reaching_goal/result',PlanningActionResult,goal_callback)

a.on_click(button_clicked_setgoal)
b.on_click(button_clicked_goalcancelled)

Insert goal x-axis


FloatText(value=0.0)

Insert goal y-axis


FloatText(value=1.0)

Plot of robot's position and goals
==========

In [16]:
import matplotlib.pyplot as plt
import rospy
#import tf
from nav_msgs.msg import Odometry
#from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation
from assignment_2_2022.msg import PlanningActionGoal

In [17]:
%matplotlib widget
class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln1, = plt.plot([], [], 'ro')
        self.ln2, = plt.plot([], [], marker='x', color='blue') 
        self.x_data, self.y_data = [] , []
        self.goal_x, self.goal_y = [] , []
    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln1
    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
    def other_callback(self, msg):
        self.goal_x.append(msg.goal.target_pose.pose.position.x)
        self.goal_y.append(msg.goal.target_pose.pose.position.y)
        
    def update_plot(self,frame):
        self.ln1.set_data(self.x_data,self.y_data)
        self.ln2.set_data(self.goal_x,self.goal_y)
        return self.ln1
    

#UNCOMMENT THIS FOR SINGLE EXECUTION
#rospy.init_node('odom_visualizer_node') 

vis = Visualiser()
sub = rospy.Subscriber('/odom',Odometry,vis.odom_callback)

sub_sub = rospy.Subscriber('/reaching_goal/goal', PlanningActionGoal, vis.other_callback)

ani=FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Laser scanner: closest obstacle
==========

In [12]:
import rospy
from sensor_msgs.msg import LaserScan
import ipywidgets as widgets

In [18]:
#this block of code gives just a simple text box with the distance of the closest obstacle
def laser_call(msg):
    arr = msg.ranges
    minimum=arr[0]
    for i in range(len(arr)):
        if arr[i] < minimum:
            minimum=arr[i]
    #print(minimum)
    w=widgets.Text(value=str(minimum),disabled=True)
    display(w)
    
    
        
#UNCOMMENT THIS FOR SINGLE EXECUTION
#rospy.init_node("laser_node")




#sub=rospy.Subscriber("/scan",LaserScan,laser_callback)

#this just gets one value for the array and then unsubscribe
msg=rospy.wait_for_message('/scan',LaserScan,timeout=5)
laser_call(msg)


Text(value='1.2350690364837646', disabled=True)

Number of goals reached and cancelled
==========

A mixed visualisation has been chosen just to make the things a little bit more complicated
for the last part of this notebook. So in the end we will have 2 sliders and a pie chart. The sliders will give the number of goal reached and cancelled whilst the pie chart the ratio between the 2. The pie chart won't be visulised if no goal has been reached/cancelled

In [14]:
import rospy
from assignment_2_2022.srv import Reached
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as pltt

In [15]:
%matplotlib widget

#UNCOMMENT THIS FOR SINGLE EXECUTION
#rospy.init_node("i_listen")


rospy.wait_for_service("/reached")
client1 = rospy.ServiceProxy("/reached",Reached)
response = client1()
goal_reached = response.reached
goal_cancelled = response.cancelled

# 2 sliders numerically important
c=widgets.IntSlider(
    value=goal_reached,
    min=0,
    max=goal_reached+goal_cancelled,
    step=1,
    description="# reached: ",
    disabled=True,
    )

d=widgets.IntSlider(
    value=goal_cancelled,
    min=0,
    max=goal_reached+goal_cancelled,
    step=1,
    description="# cancelled: ",
    disabled=True
)
display(c,d)

#We don't want to show a pie chart if that means dividing by 0 in the code... and indeed the
#pie chart wouldn't make sense in that case
if goal_reached+goal_cancelled>0:
    # A pie chart to look at the ratio
    labels = 'Reached', 'Cancelled'
    percent_r = goal_reached/(goal_reached+goal_cancelled)
    percent_c = goal_cancelled/(goal_reached+goal_cancelled)

    percent_r = percent_r*100
    percent_c = percent_c*100

    sizes = [percent_r, percent_c]
    explode=(0.0,0.0) #user can set this variable to visualize differently the pie chart
    fig1, ax1 = pltt.subplots()
    ax1.pie(sizes,explode=explode,labels=labels,autopct='%1.1f%%',shadow=True,startangle=90)
    ax1.axis('equal')
    pltt.show()

IntSlider(value=3, description='# reached: ', disabled=True, max=3)

IntSlider(value=0, description='# cancelled: ', disabled=True, max=3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …